# Setando o groq

A ideia é setar o groq para usarmos com o openai agents

In [1]:
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel, trace
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
client_groq = AsyncOpenAI(
    api_key=os.getenv("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1"
)

model_groq = OpenAIChatCompletionsModel(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    openai_client=client_groq
)

In [3]:
agente_test = Agent(
    name="Assistente",
    instructions="Você é um ótimo assistente e irá ajudar na resposta de qualquer pergunta",
    model=model_groq
)

In [4]:
result = await Runner.run(starting_agent=agente_test, input="Quem foi o campeão da nba em 2021?")

In [5]:
print(result.final_output)

O campeão da NBA em 2021 foi o Milwaukee Bucks. Eles venceram o Phoenix Suns na final da NBA de 2021, ganhando a série por 4 jogos a 2. Foi o primeiro título da NBA do Milwaukee Bucks em 50 anos. Giannis Antetokounmpo foi nomeado o MVP das Finais da NBA de 2021.


# Passo para criar o cliente, host e server MCP

Lembrando que desta forma estaremos criando o componentes do mcp via sdk openai agents

### Criando o mcp server para o fetch (que é um buscador na internet e leva o contexto para a llm)

In [6]:
from agents.mcp import MCPServerStdio

In [7]:
fetch_params = {
    "command": "uvx",
    "args": ["mcp-server-fetch"]
}

In [8]:
async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=30) as server:
    fetch_tools = await server.list_tools()

In [9]:
print(fetch_tools)

[Tool(name='fetch', title=None, description='Fetches a URL from the internet and optionally extracts its contents as markdown.\n\nAlthough originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.', inputSchema={'description': 'Parameters for fetching a URL.', 'properties': {'url': {'description': 'URL to fetch', 'format': 'uri', 'minLength': 1, 'title': 'Url', 'type': 'string'}, 'max_length': {'default': 5000, 'description': 'Maximum number of characters to return.', 'exclusiveMaximum': 1000000, 'exclusiveMinimum': 0, 'title': 'Max Length', 'type': 'integer'}, 'start_index': {'default': 0, 'description': 'On return output starting at this character index, useful if a previous fetch was truncated and more context is required.', 'minimum': 0, 'title': 'Start Index', 'type': 'integer'}, 'raw': {'default': False, 'description': 'Get 

### Criando mcp server que irá usar o playwright

In [10]:
playwright_params = {
    "command": "npx",
    "args": ["@playwright/mcp@latest"]
}

In [11]:
async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=30) as server:
    playwright_tools = await server.list_tools()

In [12]:
playwright_tools

[Tool(name='browser_close', title=None, description='Close the page', inputSchema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Close browser', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_resize', title=None, description='Resize the browser window', inputSchema={'type': 'object', 'properties': {'width': {'type': 'number', 'description': 'Width of the browser window'}, 'height': {'type': 'number', 'description': 'Height of the browser window'}}, 'required': ['width', 'height'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=ToolAnnotations(title='Resize browser window', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_console_messages', title=None, desc

### Criando mcp server de manejamento de arquivos locais

In [13]:
sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {
    "command": "npx",
    "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]
}

In [14]:
async with MCPServerStdio(params=files_params, client_session_timeout_seconds=30) as server:
    file_tools = await server.list_tools()

In [15]:
file_tools

[Tool(name='read_file', title=None, description='Read the complete contents of a file as text. DEPRECATED: Use read_text_file instead.', inputSchema={'type': 'object', 'properties': {'path': {'type': 'string'}, 'tail': {'type': 'number', 'description': 'If provided, returns only the last N lines of the file'}, 'head': {'type': 'number', 'description': 'If provided, returns only the first N lines of the file'}}, 'required': ['path'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, annotations=None, meta=None),
 Tool(name='read_text_file', title=None, description="Read the complete contents of a file from the file system as text. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Use the 'head' parameter to read only the first N lines of a file, or the 'tail' parameter to read only the last N lines of a file. Operates 

# Agora sim criando o agente com as tools

In [16]:
instrucoes = """
Você navega na internet para concluir suas instruções.
Você é altamente capaz de navegar na intenet independente de concluir a sua tarefa, você é capaz de aceitas cookies de páginas web
e outras coisas da internet. Então USE a internet.
Se o website não for te ajudar, então vá para outro. Seja persistente até conseguir realizar a tarefa que lhe foi designado.
"""

In [17]:
prompt_user = """
Procure na internet 5 restaurantes incríveis e românticos na cidade de são paulo, listando o seu preço, pontos positivos, localidade, como chegar e etc
Faça esse resumo em markdown e salve o resultado como lugares.md
"""

In [18]:
async with MCPServerStdio(params=files_params, client_session_timeout_seconds=30) as mcp_server_files:
    async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=30) as mcp_server_browser:
        agent = Agent(
            name="Você é um agente investigador de restaurantes",
            instructions=instrucoes,
            model=model_groq,
            mcp_servers=[mcp_server_files, mcp_server_browser]
        )
        with trace("Investigador gastronômico"):
            result = await Runner.run(
                starting_agent=agent,
                input=prompt_user
            )

In [19]:
print(result.final_output)

Concluí a tarefa! Foram encontrados 5 restaurantes incríveis e românticos na cidade de São Paulo, com seus preços, pontos positivos, localidades e informações de como chegar. O resultado foi salvo no arquivo `lugares.md`.
